# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

In [2]:
#!pip install scipy

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
#!unzip '/content/drive/MyDrive/Colab Notebooks/Project_data.zip' -d '/content/drive/MyDrive/Colab Notebooks/Project_data'

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

2024-03-06 06:59:22.311227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 06:59:22.311334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 06:59:22.491278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('/kaggle/input/gesture-reconition/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/kaggle/input/gesture-reconition/Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

idx_seq = range(7,26,2)
img_height = 120
img_width = 120

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
def generator(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = idx_seq # create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size, 20, img_height, img_width, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx, item in enumerate(img_idx): # Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (img_height, img_width, 3))  # Resize the image to desired dimensions

                    # crop the images and resize them. Note that the images are of 2 different shapes
                    # and the conv3D will throw an error if the inputs in a batch have different shapes

                    # batch_data[folder, idx, :, :, 0] = (image[:, :, 0] - np.percentile(image[:, :, 0], 5)) / (np.percentile(image[:, :, 0], 95) - np.percentile(image[:, :, 0], 5)) # normalise and feed in the image
                    # batch_data[folder, idx, :, :, 1] = (image[:, :, 1] - np.percentile(image[:, :, 1], 5)) / (np.percentile(image[:, :, 1], 95) - np.percentile(image[:, :, 1], 5)) # normalise and feed in the image
                    # batch_data[folder, idx, :, :, 2] = (image[:, :, 2] - np.percentile(image[:, :, 2], 5)) / (np.percentile(image[:, :, 2], 95) - np.percentile(image[:, :, 2], 5)) # normalise and feed in the image
                    batch_data[folder, idx, :, :, 0] = (image[:, :, 0]) / 255
                    batch_data[folder, idx, :, :, 1] = (image[:, :, 1]) / 255
                    batch_data[folder, idx, :, :, 2] = (image[:, :, 2]) / 255


                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels # you yield the batch_data and the batch_labels, remember what does yield do

        # handle remaining data points after full batches
        if len(folder_list) % batch_size != 0:
            remaining_batch_size = len(folder_list) % batch_size
            batch_data = np.zeros((remaining_batch_size, 20, img_height, img_height, 3))
            batch_labels = np.zeros((remaining_batch_size, 5))
            for folder in range(remaining_batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx, item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (img_height, img_width, 3))  # Resize the image to desired dimensions
                    batch_data[folder, idx, :, :, 0] = (image[:, :, 0]) / 255
                    batch_data[folder, idx, :, :, 1] = (image[:, :, 1]) / 255
                    batch_data[folder, idx, :, :, 2] = (image[:, :, 2]) / 255

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = '/kaggle/input/gesture-reconition/Project_data/train'
val_path = '/kaggle/input/gesture-reconition/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 1 # choose the number of epochs
print ('# epochs =', num_epochs)
num_classes= 5

# training sequences = 663
# validation sequences = 100
# epochs = 1


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

**Model with batch size 20**

In [9]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv3D, MaxPooling3D, MaxPooling2D, RandomFlip, RandomRotation,Rescaling, LSTM

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
#from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

def generate_model(batch_size, img_height, img_width, dropout=0.25, dense=64):
  #write your model here
  model = Sequential()

  #model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(batch_size ,img_height, img_width,3)))

  model.add(Conv3D(16, (3, 3, 3), padding='same',input_shape=(batch_size ,img_height, img_width,3)))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))

  model.add(Conv3D(32, (2, 2, 2), padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))

  model.add(Conv3D(64, (2, 2, 2), padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))


  model.add(Conv3D(128, (2, 2, 2), padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

  model.add(Flatten())
  model.add(Dense(dense,activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Dense(dense,activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Dense(num_classes,activation='softmax'))
  return model

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
model = generate_model(batch_size, img_height, img_width, 0.25, 128)

In [11]:
optimiser = 'adam' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 10, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation (Activation)     (None, 10, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization (Batch  (None, 10, 120, 120, 16   64        
 Normalization)              )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 5, 60, 60, 16)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 5, 60, 60, 32)     4

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = '/kaggle/working/' + model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

from keras.callbacks import ReduceLROnPlateau

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Validation accuracy = 0.44
Training accuracy = 0.16

Accuracy is not that great. Also 

**There is clear sign of overfitting as the difference between the training accuracy and validation accuracy is more. **

Source path =  /kaggle/input/gesture-reconition/Project_data/train ; batch size = 10
/tmp/ipykernel_34/3799537928.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
/tmp/ipykernel_34/515647629.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1709479767.022459     229 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
66/67 [============================>.] - ETA: 1s - loss: 1.5532 - categorical_accuracy: 0.4348
/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
67/67 [==============================] - ETA: 0s - loss: 1.5532 - categorical_accuracy: 0.4359Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-03-0315_29_16.612602/model-00001-1.55318-0.43590-6.49745-0.16000.h5
67/67 [==============================] - 136s 2s/step - loss: 1.5532 - categorical_accuracy: 0.4359 - val_loss: 6.4975 - val_categorical_accuracy: 0.1600 - lr: 0.0010

# **Model with Batch size 20 num of epocs 10**

In [16]:
batch_size = 20
num_epochs = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [18]:
model_2 = generate_model(batch_size, img_height, img_width, 0.25, 64)

In [19]:
optimiser = 'adam' #write your optimizer
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 20, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_4 (Activation)   (None, 20, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_6 (Bat  (None, 20, 120, 120, 16   64        
 chNormalization)            )                                   
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 10, 60, 60, 16)    0         
 g3D)                                                            
                                                                 
 conv3d_5 (Conv3D)           (None, 10, 60, 60, 32)   

In [20]:
#model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /kaggle/input/gesture-reconition/Project_data/train ; batch size = 20
/tmp/ipykernel_34/1443121102.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
/tmp/ipykernel_34/515647629.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
Epoch 1/10
33/34 [============================>.] - ETA: 2s - loss: 1.4936 - categorical_accuracy: 0.4561
/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
34/34 [==============================] - ETA: 0s - loss: 1.4920 - categorical_accuracy: 0.4555Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 20

Epoch 1: saving model to model_init_2024-03-0315_29_16.612602/model-00001-1.49202-0.45551-8.32596-0.16000.h5
34/34 [==============================] - 96s 3s/step - loss: 1.4920 - categorical_accuracy: 0.4555 - val_loss: 8.3260 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 0.9696 - categorical_accuracy: 0.5988
Epoch 2: saving model to model_init_2024-03-0315_29_16.612602/model-00002-0.96960-0.59879-19.08339-0.15000.h5
34/34 [==============================] - 89s 3s/step - loss: 0.9696 - categorical_accuracy: 0.5988 - val_loss: 19.0834 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 0.8006 - categorical_accuracy: 0.7014
Epoch 3: saving model to model_init_2024-03-0315_29_16.612602/model-00003-0.80058-0.70136-24.73187-0.16000.h5
34/34 [==============================] - 85s 3s/step - loss: 0.8006 - categorical_accuracy: 0.7014 - val_loss: 24.7319 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 4/10
34/34 [==============================] - ETA: 0s - loss: 0.7110 - categorical_accuracy: 0.7360
Epoch 4: saving model to model_init_2024-03-0315_29_16.612602/model-00004-0.71102-0.73605-33.21298-0.11000.h5
34/34 [==============================] - 81s 2s/step - loss: 0.7110 - categorical_accuracy: 0.7360 - val_loss: 33.2130 - val_categorical_accuracy: 0.1100 - lr: 0.0010
Epoch 5/10
34/34 [==============================] - ETA: 0s - loss: 0.7553 - categorical_accuracy: 0.7376
Epoch 5: saving model to model_init_2024-03-0315_29_16.612602/model-00005-0.75534-0.73756-25.44184-0.15000.h5
34/34 [==============================] - 81s 2s/step - loss: 0.7553 - categorical_accuracy: 0.7376 - val_loss: 25.4418 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 6/10
34/34 [==============================] - ETA: 0s - loss: 0.6312 - categorical_accuracy: 0.7707
Epoch 6: saving model to model_init_2024-03-0315_29_16.612602/model-00006-0.63116-0.77074-16.28077-0.16000.h5
34/34 [==============================] - 80s 2s/step - loss: 0.6312 - categorical_accuracy: 0.7707 - val_loss: 16.2808 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 7/10
34/34 [==============================] - ETA: 0s - loss: 0.5374 - categorical_accuracy: 0.8100
Epoch 7: saving model to model_init_2024-03-0315_29_16.612602/model-00007-0.53740-0.80995-12.35917-0.17000.h5
34/34 [==============================] - 78s 2s/step - loss: 0.5374 - categorical_accuracy: 0.8100 - val_loss: 12.3592 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 8/10
34/34 [==============================] - ETA: 0s - loss: 0.5046 - categorical_accuracy: 0.8115
Epoch 8: saving model to model_init_2024-03-0315_29_16.612602/model-00008-0.50459-0.81146-10.81409-0.13000.h5
34/34 [==============================] - 79s 2s/step - loss: 0.5046 - categorical_accuracy: 0.8115 - val_loss: 10.8141 - val_categorical_accuracy: 0.1300 - lr: 0.0010
Epoch 9/10
34/34 [==============================] - ETA: 0s - loss: 0.4961 - categorical_accuracy: 0.8069
Epoch 9: saving model to model_init_2024-03-0315_29_16.612602/model-00009-0.49606-0.80694-11.32586-0.17000.h5
34/34 [==============================] - 80s 2s/step - loss: 0.4961 - categorical_accuracy: 0.8069 - val_loss: 11.3259 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 10/10
34/34 [==============================] - ETA: 0s - loss: 0.4405 - categorical_accuracy: 0.8386
Epoch 10: saving model to model_init_2024-03-0315_29_16.612602/model-00010-0.44049-0.83861-7.41945-0.17000.h5
34/34 [==============================] - 80s 2s/step - loss: 0.4405 - categorical_accuracy: 0.8386 - val_loss: 7.4195 - val_categorical_accuracy: 0.1700 - lr: 0.0010
<keras.src.callbacks.History at 0x7f9af809e1a0>

**There is clear overfitting in the above model due to a large difference between training and Validation accuracy**

# Model 3 - Batch size 15, epochs 30, drop out 0.25 and dense 64

In [21]:
batch_size = 15
num_epochs = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [22]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [23]:
model_3 = generate_model(batch_size, img_height, img_width, 0.25, 64)

In [24]:
optimiser = 'adam' #write your optimizer
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_3.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 15, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_8 (Activation)   (None, 15, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_12 (Ba  (None, 15, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 max_pooling3d_8 (MaxPoolin  (None, 7, 60, 60, 16)     0         
 g3D)                                                            
                                                                 
 conv3d_9 (Conv3D)           (None, 7, 60, 60, 32)    

In [25]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_34/3490547937.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
/tmp/ipykernel_34/515647629.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /kaggle/input/gesture-reconition/Project_data/train ; batch size = 15
Epoch 1/30


I0000 00:00:1709708413.991369     113 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/45 [============================>.] - ETA: 1s - loss: 1.5095 - categorical_accuracy: 0.4182

/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


45/45 [==============================] - ETA: 0s - loss: 1.5135 - categorical_accuracy: 0.4163Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 15

Epoch 1: saving model to /kaggle/working/model_init_2024-03-0606_59_34.127778/model-00001-1.51352-0.41629-8.48297-0.16000.h5
45/45 [==============================] - 108s 2s/step - loss: 1.5135 - categorical_accuracy: 0.4163 - val_loss: 8.4830 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/30


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


45/45 [==============================] - ETA: 0s - loss: 1.0780 - categorical_accuracy: 0.5867
Epoch 2: saving model to /kaggle/working/model_init_2024-03-0606_59_34.127778/model-00002-1.07799-0.58673-13.90831-0.13000.h5
45/45 [==============================] - 76s 2s/step - loss: 1.0780 - categorical_accuracy: 0.5867 - val_loss: 13.9083 - val_categorical_accuracy: 0.1300 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - ETA: 0s - loss: 1.1999 - categorical_accuracy: 0.5385
Epoch 3: saving model to /kaggle/working/model_init_2024-03-0606_59_34.127778/model-00003-1.19994-0.53846-10.64747-0.18000.h5
45/45 [==============================] - 79s 2s/step - loss: 1.1999 - categorical_accuracy: 0.5385 - val_loss: 10.6475 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 4/30
45/45 [==============================] - ETA: 0s - loss: 0.9741 - categorical_accuracy: 0.6229
Epoch 4: saving model to /kaggle/working/model_init_2024-03-0606_59_34.127778/model-00004-0.97415-0.62293-7

Source path = /content/drive/MyDrive/Colab NotebooksEpoch 1/30
44/45 [============================>.] - ETA: 1s - loss: 1.5154 - categorical_accuracy: 0.4288
/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
45/45 [==============================] - ETA: 0s - loss: 1.5175 - categorical_accuracy: 0.4284Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 15

Epoch 1: saving model to model_init_2024-03-0315_29_16.612602/model-00001-1.51751-0.42836-3.96369-0.19000.h5
45/45 [==============================] - 98s 2s/step - loss: 1.5175 - categorical_accuracy: 0.4284 - val_loss: 3.9637 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 2/30
45/45 [==============================] - ETA: 0s - loss: 1.1068 - categorical_accuracy: 0.5686
Epoch 2: saving model to model_init_2024-03-0315_29_16.612602/model-00002-1.10676-0.56863-9.09869-0.16000.h5
45/45 [==============================] - 94s 2s/step - loss: 1.1068 - categorical_accuracy: 0.5686 - val_loss: 9.0987 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - ETA: 0s - loss: 0.9209 - categorical_accuracy: 0.6410
Epoch 3: saving model to model_init_2024-03-0315_29_16.612602/model-00003-0.92094-0.64103-9.00184-0.16000.h5
45/45 [==============================] - 84s 2s/step - loss: 0.9209 - categorical_accuracy: 0.6410 - val_loss: 9.0018 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 4/30
45/45 [==============================] - ETA: 0s - loss: 0.7995 - categorical_accuracy: 0.6802
Epoch 4: saving model to model_init_2024-03-0315_29_16.612602/model-00004-0.79947-0.68024-8.35011-0.19000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.7995 - categorical_accuracy: 0.6802 - val_loss: 8.3501 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 5/30
45/45 [==============================] - ETA: 0s - loss: 0.6905 - categorical_accuracy: 0.7300
Epoch 5: saving model to model_init_2024-03-0315_29_16.612602/model-00005-0.69049-0.73002-9.37489-0.15000.h5
45/45 [==============================] - 82s 2s/step - loss: 0.6905 - categorical_accuracy: 0.7300 - val_loss: 9.3749 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 6/30
45/45 [==============================] - ETA: 0s - loss: 0.6135 - categorical_accuracy: 0.7677
Epoch 6: saving model to model_init_2024-03-0315_29_16.612602/model-00006-0.61346-0.76772-10.84826-0.16000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.6135 - categorical_accuracy: 0.7677 - val_loss: 10.8483 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 7/30
45/45 [==============================] - ETA: 0s - loss: 0.6080 - categorical_accuracy: 0.7888
Epoch 7: saving model to model_init_2024-03-0315_29_16.612602/model-00007-0.60800-0.78884-7.66915-0.30000.h5
45/45 [==============================] - 73s 2s/step - loss: 0.6080 - categorical_accuracy: 0.7888 - val_loss: 7.6691 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 8/30
45/45 [==============================] - ETA: 0s - loss: 0.6045 - categorical_accuracy: 0.7798
Epoch 8: saving model to model_init_2024-03-0315_29_16.612602/model-00008-0.60450-0.77979-8.06716-0.31000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.6045 - categorical_accuracy: 0.7798 - val_loss: 8.0672 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 9/30
45/45 [==============================] - ETA: 0s - loss: 0.5257 - categorical_accuracy: 0.7949
Epoch 9: saving model to model_init_2024-03-0315_29_16.612602/model-00009-0.52573-0.79487-5.38186-0.32000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.5257 - categorical_accuracy: 0.7949 - val_loss: 5.3819 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 10/30
45/45 [==============================] - ETA: 0s - loss: 0.6150 - categorical_accuracy: 0.7511
Epoch 10: saving model to model_init_2024-03-0315_29_16.612602/model-00010-0.61499-0.75113-3.45167-0.31000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.6150 - categorical_accuracy: 0.7511 - val_loss: 3.4517 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 11/30
45/45 [==============================] - ETA: 0s - loss: 0.4702 - categorical_accuracy: 0.8190
Epoch 11: saving model to model_init_2024-03-0315_29_16.612602/model-00011-0.47024-0.81900-2.56021-0.30000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.4702 - categorical_accuracy: 0.8190 - val_loss: 2.5602 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 12/30
45/45 [==============================] - ETA: 0s - loss: 0.3700 - categorical_accuracy: 0.8431
Epoch 12: saving model to model_init_2024-03-0315_29_16.612602/model-00012-0.36999-0.84314-3.56849-0.40000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.3700 - categorical_accuracy: 0.8431 - val_loss: 3.5685 - val_categorical_accuracy: 0.4000 - lr: 0.0010
Epoch 13/30
45/45 [==============================] - ETA: 0s - loss: 0.3666 - categorical_accuracy: 0.8733
Epoch 13: saving model to model_init_2024-03-0315_29_16.612602/model-00013-0.36664-0.87330-3.73233-0.36000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.3666 - categorical_accuracy: 0.8733 - val_loss: 3.7323 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 14/30
45/45 [==============================] - ETA: 0s - loss: 0.3802 - categorical_accuracy: 0.8748
Epoch 14: saving model to model_init_2024-03-0315_29_16.612602/model-00014-0.38018-0.87481-1.32997-0.60000.h5
45/45 [==============================] - 75s 2s/step - loss: 0.3802 - categorical_accuracy: 0.8748 - val_loss: 1.3300 - val_categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 15/30
45/45 [==============================] - ETA: 0s - loss: 0.2904 - categorical_accuracy: 0.8989
Epoch 15: saving model to model_init_2024-03-0315_29_16.612602/model-00015-0.29043-0.89894-1.61801-0.55000.h5
45/45 [==============================] - 82s 2s/step - loss: 0.2904 - categorical_accuracy: 0.8989 - val_loss: 1.6180 - val_categorical_accuracy: 0.5500 - lr: 0.0010
Epoch 16/30
45/45 [==============================] - ETA: 0s - loss: 0.3079 - categorical_accuracy: 0.8839
Epoch 16: saving model to model_init_2024-03-0315_29_16.612602/model-00016-0.30795-0.88386-1.49565-0.64000.h5
45/45 [==============================] - 79s 2s/step - loss: 0.3079 - categorical_accuracy: 0.8839 - val_loss: 1.4956 - val_categorical_accuracy: 0.6400 - lr: 0.0010
Epoch 17/30
45/45 [==============================] - ETA: 0s - loss: 0.3993 - categorical_accuracy: 0.8582
Epoch 17: saving model to model_init_2024-03-0315_29_16.612602/model-00017-0.39926-0.85822-2.18048-0.51000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.3993 - categorical_accuracy: 0.8582 - val_loss: 2.1805 - val_categorical_accuracy: 0.5100 - lr: 0.0010
Epoch 18/30
45/45 [==============================] - ETA: 0s - loss: 0.2736 - categorical_accuracy: 0.9020
Epoch 18: saving model to model_init_2024-03-0315_29_16.612602/model-00018-0.27361-0.90196-1.57163-0.62000.h5
45/45 [==============================] - 72s 2s/step - loss: 0.2736 - categorical_accuracy: 0.9020 - val_loss: 1.5716 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 19/30
45/45 [==============================] - ETA: 0s - loss: 0.2063 - categorical_accuracy: 0.9246
Epoch 19: saving model to model_init_2024-03-0315_29_16.612602/model-00019-0.20633-0.92459-1.63832-0.62000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.2063 - categorical_accuracy: 0.9246 - val_loss: 1.6383 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 20/30
45/45 [==============================] - ETA: 0s - loss: 0.2017 - categorical_accuracy: 0.9276
Epoch 20: saving model to model_init_2024-03-0315_29_16.612602/model-00020-0.20169-0.92760-1.09318-0.65000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.2017 - categorical_accuracy: 0.9276 - val_loss: 1.0932 - val_categorical_accuracy: 0.6500 - lr: 0.0010
Epoch 21/30
45/45 [==============================] - ETA: 0s - loss: 0.4429 - categorical_accuracy: 0.8597
Epoch 21: saving model to model_init_2024-03-0315_29_16.612602/model-00021-0.44287-0.85973-2.90883-0.41000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.4429 - categorical_accuracy: 0.8597 - val_loss: 2.9088 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 22/30
45/45 [==============================] - ETA: 0s - loss: 0.3324 - categorical_accuracy: 0.8929
Epoch 22: saving model to model_init_2024-03-0315_29_16.612602/model-00022-0.33244-0.89291-1.48433-0.50000.h5
45/45 [==============================] - 79s 2s/step - loss: 0.3324 - categorical_accuracy: 0.8929 - val_loss: 1.4843 - val_categorical_accuracy: 0.5000 - lr: 0.0010
Epoch 23/30
45/45 [==============================] - ETA: 0s - loss: 0.3622 - categorical_accuracy: 0.8522
Epoch 23: saving model to model_init_2024-03-0315_29_16.612602/model-00023-0.36221-0.85219-1.54422-0.59000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.3622 - categorical_accuracy: 0.8522 - val_loss: 1.5442 - val_categorical_accuracy: 0.5900 - lr: 0.0010
Epoch 24/30
45/45 [==============================] - ETA: 0s - loss: 0.2240 - categorical_accuracy: 0.9321
Epoch 24: saving model to model_init_2024-03-0315_29_16.612602/model-00024-0.22400-0.93213-0.93862-0.66000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.2240 - categorical_accuracy: 0.9321 - val_loss: 0.9386 - val_categorical_accuracy: 0.6600 - lr: 0.0010
Epoch 25/30
45/45 [==============================] - ETA: 0s - loss: 0.3144 - categorical_accuracy: 0.8944
Epoch 25: saving model to model_init_2024-03-0315_29_16.612602/model-00025-0.31442-0.89442-0.66863-0.73000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.3144 - categorical_accuracy: 0.8944 - val_loss: 0.6686 - val_categorical_accuracy: 0.7300 - lr: 0.0010
Epoch 26/30
45/45 [==============================] - ETA: 0s - loss: 0.3299 - categorical_accuracy: 0.8869
Epoch 26: saving model to model_init_2024-03-0315_29_16.612602/model-00026-0.32987-0.88688-1.06794-0.68000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.3299 - categorical_accuracy: 0.8869 - val_loss: 1.0679 - val_categorical_accuracy: 0.6800 - lr: 0.0010
Epoch 27/30
45/45 [==============================] - ETA: 0s - loss: 0.3082 - categorical_accuracy: 0.8959
Epoch 27: saving model to model_init_2024-03-0315_29_16.612602/model-00027-0.30821-0.89593-1.30123-0.62000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.3082 - categorical_accuracy: 0.8959 - val_loss: 1.3012 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 28/30
45/45 [==============================] - ETA: 0s - loss: 0.2952 - categorical_accuracy: 0.9020
Epoch 28: saving model to model_init_2024-03-0315_29_16.612602/model-00028-0.29516-0.90196-2.16103-0.57000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.2952 - categorical_accuracy: 0.9020 - val_loss: 2.1610 - val_categorical_accuracy: 0.5700 - lr: 0.0010
Epoch 29/30
45/45 [==============================] - ETA: 0s - loss: 0.4707 - categorical_accuracy: 0.8265
Epoch 29: saving model to model_init_2024-03-0315_29_16.612602/model-00029-0.47074-0.82655-2.83704-0.36000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.4707 - categorical_accuracy: 0.8265 - val_loss: 2.8370 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 30/30
45/45 [==============================] - ETA: 0s - loss: 0.4537 - categorical_accuracy: 0.8341
Epoch 30: saving model to model_init_2024-03-0315_29_16.612602/model-00030-0.45372-0.83409-1.29258-0.53000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.4537 - categorical_accuracy: 0.8341 - val_loss: 1.2926 - val_categorical_accuracy: 0.5300 - lr: 0.0010
<keras.src.callbacks.History at 0x7f9aac1151e0>/Project_data/Project_data/train ; batch size = 15 :1: UserWarning: Model.fit_generator is deprecated and will be removed in a future version. Please use Model.fit, which supports generators. model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0) :13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use import imageio.v2 as imageio or call imageio.v2.imread directly. image = imread(source_path+'/'+ t[folder + (batchbatch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32) Epoch 1/30 44/45 [============================>.] - ETA: 1s - loss: 1.4500 - categorical_accuracy: 0.4727:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use import imageio.v2 as imageio or call imageio.v2.imread directly. image = imread(source_path+'/'+ t[folder + (num_batchesbatch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32) 45/45 [==============================] - ETA: 0s - loss: 1.4532 - categorical_accuracy: 0.4721Source path = /content/drive/MyDrive/Colab Notebooks/Project_data/Project_data/val ; batch size = 15

Epoch 1: saving model to model_init_2024-03-0312_35_43.575611/model-00001-1.45318-0.47210-7.98422-0.16000.h5 45/45 [==============================] - 118s 2s/step - loss: 1.4532 - categorical_accuracy: 0.4721 - val_loss: 7.9842 - val_categorical_accuracy: 0.1600 - lr: 0.0010 Epoch 2/30 /usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via model.save(). This file format is considered legacy. We recommend using instead the native Keras format, e.g. model.save('my_model.keras'). saving_api.save_model( 45/45 [==============================] - ETA: 0s - loss: 0.9139 - categorical_accuracy: 0.6486 
Epoch 2: saving model to model_init_2024-03-0312_35_43.575611/model-00002-0.91391-0.64857-16.36342-0.13000.h5 45/45 [==============================] - 99s 2s/step - loss: 0.9139 - categorical_accuracy: 0.6486 - val_loss: 16.3634 - val_categorical_accuracy: 0.1300 - lr: 0.0010 Epoch 3/30 45/45 [==============================] - ETA: 0s - loss: 0.8474 - categorical_accuracy: 0.6456 
Epoch 3: saving model to model_init_2024-03-0312_35_43.575611/model-00003-0.84739-0.64555-16.90599-0.18000.h5 45/45 [==============================] - 98s 2s/step - loss: 0.8474 - categorical_accuracy: 0.6456 - val_loss: 16.9060 - val_categorical_accuracy: 0.1800 - lr: 0.0010 Epoch 4/30 45/45 [==============================] - ETA: 0s - loss: 0.7686 - categorical_accuracy: 0.7164 
Epoch 4: saving model to model_init_2024-03-0312_35_43.575611/model-00004-0.76862-0.71644-21.85694-0.17000.h5 45/45 [==============================] - 102s 2s/step - loss: 0.7686 - categorical_accuracy: 0.7164 - val_loss: 21.8569 - val_categorical_accuracy: 0.1700 - lr: 0.0010 Epoch 5/30 45/45 [==============================] - ETA: 0s - loss: 0.5994 - categorical_accuracy: 0.7979 
Epoch 5: saving model to model_init_2024-03-0312_35_43.575611/model-00005-0.59942-0.79789-25.26162-0.17000.h5 45/45 [==============================] - 101s 2s/step - loss: 0.5994 - categorical_accuracy: 0.7979 - val_loss: 25.2616 - val_categorical_accuracy: 0.1700 - lr: 0.0010 Epoch 6/30 45/45 [==============================] - ETA: 0s - loss: 0.6081 - categorical_accuracy: 0.7934 
Epoch 6: saving model to model_init_2024-03-0312_35_43.575611/model-00006-0.60806-0.79336-14.68379-0.14000.h5 45/45 [==============================] - 100s 2s/step - loss: 0.6081 - categorical_accuracy: 0.7934 - val_loss: 14.6838 - val_categorical_accuracy: 0.1400 - lr: 0.0010 Epoch 7/30 45/45 [==============================] - ETA: 0s - loss: 0.4069 - categorical_accuracy: 0.8582 
Epoch 7: saving model to model_init_2024-03-0312_35_43.575611/model-00007-0.40690-0.85822-10.88318-0.31000.h5 45/45 [==============================] - 101s 2s/step - loss: 0.4069 - categorical_accuracy: 0.8582 - val_loss: 10.8832 - val_categorical_accuracy: 0.3100 - lr: 0.0010 Epoch 8/30 45/45 [==============================] - ETA: 0s - loss: 0.4059 - categorical_accuracy: 0.8643 
Epoch 8: saving model to model_init_2024-03-0312_35_43.575611/model-00008-0.40593-0.86425-9.67690-0.23000.h5 45/45 [==============================] - 98s 2s/step - loss: 0.4059 - categorical_accuracy: 0.8643 - val_loss: 9.6769 - val_categorical_accuracy: 0.2300 - lr: 0.0010 Epoch 9/30 45/45 [==============================] - ETA: 0s - loss: 0.4068 - categorical_accuracy: 0.8492 
Epoch 9: saving model to model_init_2024-03-0312_35_43.575611/model-00009-0.40679-0.84917-5.77921-0.20000.h5 45/45 [==============================] - 102s 2s/step - loss: 0.4068 - categorical_accuracy: 0.8492 - val_loss: 5.7792 - val_categorical_accuracy: 0.2000 - lr: 0.0010 Epoch 10/30 45/45 [==============================] - ETA: 0s - loss: 0.4271 - categorical_accuracy: 0.8552 
Epoch 10: saving model to model_init_2024-03-0312_35_43.575611/model-00010-0.42709-0.85520-2.95347-0.25000.h5 45/45 [==============================] - 103s 2s/step - loss: 0.4271 - categorical_accuracy: 0.8552 - val_loss: 2.9535 - val_categorical_accuracy: 0.2500 - lr: 0.0010 Epoch 11/30 45/45 [==============================] - ETA: 0s - loss: 0.3398 - categorical_accuracy: 0.8643 
Epoch 11: saving model to model_init_2024-03-0312_35_43.575611/model-00011-0.33984-0.86425-4.72249-0.19000.h5 45/45 [==============================] - 102s 2s/step - loss: 0.3398 - categorical_accuracy: 0.8643 - val_loss: 4.7225 - val_categorical_accuracy: 0.1900 - lr: 0.0010 Epoch 12/30 45/45 [==============================] - ETA: 0s - loss: 0.3317 - categorical_accuracy: 0.8688 
Epoch 12: saving model to model_init_2024-03-0312_35_43.575611/model-00012-0.33172-0.86878-2.51660-0.38000.h5 45/45 [==============================] - 101s 2s/step - loss: 0.3317 - categorical_accuracy: 0.8688 - val_loss: 2.5166 - val_categorical_accuracy: 0.3800 - lr: 0.0010 Epoch 13/30 45/45 [==============================] - ETA: 0s - loss: 0.2177 - categorical_accuracy: 0.9351 
Epoch 13: saving model to model_init_2024-03-0312_35_43.575611/model-00013-0.21771-0.93514-0.92246-0.70000.h5 45/45 [==============================] - 99s 2s/step - loss: 0.2177 - categorical_accuracy: 0.9351 - val_loss: 0.9225 - val_categorical_accuracy: 0.7000 - lr: 0.0010 Epoch 14/30 45/45 [==============================] - ETA: 0s - loss: 0.1922 - categorical_accuracy: 0.9412 
Epoch 14: saving model to model_init_2024-03-0312_35_43.575611/model-00014-0.19220-0.94118-1.27620-0.50000.h5 45/45 [==============================] - 103s 2s/step - loss: 0.1922 - categorical_accuracy: 0.9412 - val_loss: 1.2762 - val_categorical_accuracy: 0.5000 - lr: 0.0010 Epoch 15/30 45/45 [==============================] - ETA: 0s - loss: 0.2002 - categorical_accuracy: 0.9336 
Epoch 15: saving model to model_init_2024-03-0312_35_43.575611/model-00015-0.20021-0.93363-1.25185-0.56000.h5 45/45 [==============================] - 97s 2s/step - loss: 0.2002 - categorical_accuracy: 0.9336 - val_loss: 1.2519 - val_categorical_accuracy: 0.5600 - lr: 0.0010 Epoch 16/30 45/45 [==============================] - ETA: 0s - loss: 0.4257 - categorical_accuracy: 0.8386 
Epoch 16: saving model to model_init_2024-03-0312_35_43.575611/model-00016-0.42567-0.83861-20.58014-0.22000.h5 45/45 [==============================] - 95s 2s/step - loss: 0.4257 - categorical_accuracy: 0.8386 - val_loss: 20.5801 - val_categorical_accuracy: 0.2200 - lr: 0.0010 Epoch 17/30 45/45 [==============================] - ETA: 0s - loss: 0.4324 - categorical_accuracy: 0.8356 
Epoch 17: saving model to model_init_2024-03-0312_35_43.575611/model-00017-0.43239-0.83560-4.51925-0.55000.h5 45/45 [==============================] - 96s 2s/step - loss: 0.4324 - categorical_accuracy: 0.8356 - val_loss: 4.5192 - val_categorical_accuracy: 0.5500 - lr: 0.0010 Epoch 18/30 45/45 [==============================] - ETA: 0s - loss: 0.3062 - categorical_accuracy: 0.8884 
Epoch 18: saving model to model_init_2024-03-0312_35_43.575611/model-00018-0.30618-0.88839-0.86285-0.72000.h5 45/45 [==============================] - 95s 2s/step - loss: 0.3062 - categorical_accuracy: 0.8884 - val_loss: 0.8629 - val_categorical_accuracy: 0.7200 - lr: 0.0010 Epoch 19/30 45/45 [==============================] - ETA: 0s - loss: 0.2185 - categorical_accuracy: 0.9276 
Epoch 19: saving model to model_init_2024-03-0312_35_43.575611/model-00019-0.21849-0.92760-2.18978-0.46000.h5 45/45 [==============================] - 98s 2s/step - loss: 0.2185 - categorical_accuracy: 0.9276 - val_loss: 2.1898 - val_categorical_accuracy: 0.4600 - lr: 0.0010 Epoch 20/30 45/45 [==============================] - ETA: 0s - loss: 0.2089 - categorical_accuracy: 0.9412 
Epoch 20: saving model to model_init_2024-03-0312_35_43.575611/model-00020-0.20894-0.94118-1.77654-0.51000.h5 45/45 [==============================] - 100s 2s/step - loss: 0.2089 - categorical_accuracy: 0.9412 - val_loss: 1.7765 - val_categorical_accuracy: 0.5100 - lr: 0.0010 Epoch 21/30 45/45 [==============================] - ETA: 0s - loss: 0.2961 - categorical_accuracy: 0.8974 
Epoch 21: saving model to model_init_2024-03-0312_35_43.575611/model-00021-0.29606-0.89744-0.49252-0.83000.h5 45/45 [==============================] - 97s 2s/step - loss: 0.2961 - categorical_accuracy: 0.8974 - val_loss: 0.4925 - val_categorical_accuracy: 0.8300 - lr: 0.0010 Epoch 22/30 45/45 [==============================] - ETA: 0s - loss: 0.2034 - categorical_accuracy: 0.9291 
Epoch 22: saving model to model_init_2024-03-0312_35_43.575611/model-00022-0.20338-0.92911-0.79607-0.73000.h5 45/45 [==============================] - 101s 2s/step - loss: 0.2034 - categorical_accuracy: 0.9291 - val_loss: 0.7961 - val_categorical_accuracy: 0.7300 - lr: 0.0010 Epoch 23/30 45/45 [==============================] - ETA: 0s - loss: 0.1754 - categorical_accuracy: 0.9397 
Epoch 23: saving model to model_init_2024-03-0312_35_43.575611/model-00023-0.17538-0.93967-1.16336-0.74000.h5 45/45 [==============================] - 99s 2s/step - loss: 0.1754 - categorical_accuracy: 0.9397 - val_loss: 1.1634 - val_categorical_accuracy: 0.7400 - lr: 0.0010 Epoch 24/30 45/45 [==============================] - ETA: 0s - loss: 0.2523 - categorical_accuracy: 0.9050 
Epoch 24: saving model to model_init_2024-03-0312_35_43.575611/model-00024-0.25229-0.90498-1.20203-0.72000.h5 45/45 [==============================] - 99s 2s/step - loss: 0.2523 - categorical_accuracy: 0.9050 - val_loss: 1.2020 - val_categorical_accuracy: 0.7200 - lr: 0.0010 Epoch 25/30 45/45 [==============================] - ETA: 0s - loss: 0.1711 - categorical_accuracy: 0.9351 
Epoch 25: saving model to model_init_2024-03-0312_35_43.575611/model-00025-0.17110-0.93514-0.85999-0.71000.h5 45/45 [==============================] - 97s 2s/step - loss: 0.1711 - categorical_accuracy: 0.9351 - val_loss: 0.8600 - val_categorical_accuracy: 0.7100 - lr: 0.0010 Epoch 26/30 45/45 [==============================] - ETA: 0s - loss: 0.1236 - categorical_accuracy: 0.9593 
Epoch 26: saving model to model_init_2024-03-0312_35_43.575611/model-00026-0.12363-0.95928-0.50714-0.81000.h5 45/45 [==============================] - 100s 2s/step - loss: 0.1236 - categorical_accuracy: 0.9593 - val_loss: 0.5071 - val_categorical_accuracy: 0.8100 - lr: 0.0010 Epoch 27/30 45/45 [==============================] - ETA: 0s - loss: 0.1060 - categorical_accuracy: 0.9668 
Epoch 27: saving model to model_init_2024-03-0312_35_43.575611/model-00027-0.10599-0.96682-0.82720-0.74000.h5 45/45 [==============================] - 97s 2s/step - loss: 0.1060 - categorical_accuracy: 0.9668 - val_loss: 0.8272 - val_categorical_accuracy: 0.7400 - lr: 0.0010 Epoch 28/30 45/45 [==============================] - ETA: 0s - loss: 0.0961 - categorical_accuracy: 0.9683 
Epoch 28: saving model to model_init_2024-03-0312_35_43.575611/model-00028-0.09608-0.96833-0.46425-0.83000.h5 45/45 [==============================] - 100s 2s/step - loss: 0.0961 - categorical_accuracy: 0.9683 - val_loss: 0.4643 - val_categorical_accuracy: 0.8300 - lr: 0.0010 Epoch 29/30 45/45 [==============================] - ETA: 0s - loss: 0.0985 - categorical_accuracy: 0.9713 
Epoch 29: saving model to model_init_2024-03-0312_35_43.575611/model-00029-0.09850-0.97134-0.32871-0.88000.h5 45/45 [==============================] - 98s 2s/step - loss: 0.0985 - categorical_accuracy: 0.9713 - val_loss: 0.3287 - val_categorical_accuracy: 0.8800 - lr: 0.0010 Epoch 30/30 45/45 [==============================] - ETA: 0s - loss: 0.1892 - categorical_accuracy: 0.9382 
Epoch 30: saving model to model_init_2024-03-0312_35_43.575611/model-00030-0.18919-0.93816-1.53691-0.70000.h5 45/45 [==============================] - 102s 2s/step - loss: 0.1892 - categorical_accuracy: 0.9382 - val_loss: 1.5369 - val_categorical_accuracy: 0.7000 - lr: 0.0010 <keras.src.callbacks.History at 0x7fa700406e30>

The best validation accuracy is 88% and training accuracy of 97%. The difference is lesser now compared to the previous models. 

# Model 4 - Batch 15, epochs 30, dropout 0.4

In [26]:
batch_size = 15
num_epochs = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [27]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [28]:
model_4 = generate_model(batch_size, img_height, img_width, 0.4, 128)

In [30]:
optimiser = 'sgd' #write your optimizer
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_4.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 15, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_12 (Activation)  (None, 15, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_18 (Ba  (None, 15, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 max_pooling3d_12 (MaxPooli  (None, 7, 60, 60, 16)     0         
 ng3D)                                                           
                                                                 
 conv3d_13 (Conv3D)          (None, 7, 60, 60, 32)    

In [31]:
model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_34/3008718817.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
/tmp/ipykernel_34/515647629.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /kaggle/input/gesture-reconition/Project_data/train ; batch size = 15
Epoch 1/30
44/45 [============================>.] - ETA: 2s - loss: 1.9614 - categorical_accuracy: 0.3591

/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


45/45 [==============================] - ETA: 0s - loss: 1.9567 - categorical_accuracy: 0.3590Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 15

Epoch 1: saving model to /kaggle/working/model_init_2024-03-0606_59_34.127778/model-00001-1.95670-0.35897-2.32275-0.21000.h5
45/45 [==============================] - 153s 3s/step - loss: 1.9567 - categorical_accuracy: 0.3590 - val_loss: 2.3228 - val_categorical_accuracy: 0.2100 - lr: 0.0100
Epoch 2/30
45/45 [==============================] - ETA: 0s - loss: 1.4882 - categorical_accuracy: 0.4434
Epoch 2: saving model to /kaggle/working/model_init_2024-03-0606_59_34.127778/model-00002-1.48820-0.44344-5.53153-0.19000.h5
45/45 [==============================] - 102s 2s/step - loss: 1.4882 - categorical_accuracy: 0.4434 - val_loss: 5.5315 - val_categorical_accuracy: 0.1900 - lr: 0.0100
Epoch 3/30
45/45 [==============================] - ETA: 0s - loss: 1.3266 - categorical_accuracy: 0.5128
Epoch 3: saving model to /

Epoch 1/30
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1709552587.359955     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
44/45 [============================>.] - ETA: 2s - loss: 1.8382 - categorical_accuracy: 0.3909
/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
45/45 [==============================] - ETA: 0s - loss: 1.8428 - categorical_accuracy: 0.3906Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 15

Epoch 1: saving model to model_init_2024-03-0411_42_54.002711/model-00001-1.84276-0.39065-9.01466-0.21000.h5
45/45 [==============================] - 154s 3s/step - loss: 1.8428 - categorical_accuracy: 0.3906 - val_loss: 9.0147 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 2/30
/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
45/45 [==============================] - ETA: 0s - loss: 1.3755 - categorical_accuracy: 0.5023
Epoch 2: saving model to model_init_2024-03-0411_42_54.002711/model-00002-1.37554-0.50226-18.28967-0.20000.h5
45/45 [==============================] - 87s 2s/step - loss: 1.3755 - categorical_accuracy: 0.5023 - val_loss: 18.2897 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - ETA: 0s - loss: 1.1374 - categorical_accuracy: 0.5792
Epoch 3: saving model to model_init_2024-03-0411_42_54.002711/model-00003-1.13741-0.57919-24.65601-0.24000.h5
45/45 [==============================] - 89s 2s/step - loss: 1.1374 - categorical_accuracy: 0.5792 - val_loss: 24.6560 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 4/30
45/45 [==============================] - ETA: 0s - loss: 0.9714 - categorical_accuracy: 0.6305
Epoch 4: saving model to model_init_2024-03-0411_42_54.002711/model-00004-0.97140-0.63047-27.72349-0.21000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.9714 - categorical_accuracy: 0.6305 - val_loss: 27.7235 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 5/30
45/45 [==============================] - ETA: 0s - loss: 1.1433 - categorical_accuracy: 0.5777
Epoch 5: saving model to model_init_2024-03-0411_42_54.002711/model-00005-1.14333-0.57768-16.12456-0.21000.h5
45/45 [==============================] - 77s 2s/step - loss: 1.1433 - categorical_accuracy: 0.5777 - val_loss: 16.1246 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 6/30
45/45 [==============================] - ETA: 0s - loss: 1.1868 - categorical_accuracy: 0.5732
Epoch 6: saving model to model_init_2024-03-0411_42_54.002711/model-00006-1.18684-0.57315-8.31426-0.21000.h5
45/45 [==============================] - 84s 2s/step - loss: 1.1868 - categorical_accuracy: 0.5732 - val_loss: 8.3143 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 7/30
45/45 [==============================] - ETA: 0s - loss: 0.9198 - categorical_accuracy: 0.6727
Epoch 7: saving model to model_init_2024-03-0411_42_54.002711/model-00007-0.91984-0.67270-8.25928-0.27000.h5
45/45 [==============================] - 75s 2s/step - loss: 0.9198 - categorical_accuracy: 0.6727 - val_loss: 8.2593 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 8/30
45/45 [==============================] - ETA: 0s - loss: 0.9736 - categorical_accuracy: 0.6154
Epoch 8: saving model to model_init_2024-03-0411_42_54.002711/model-00008-0.97363-0.61538-6.37939-0.24000.h5
45/45 [==============================] - 79s 2s/step - loss: 0.9736 - categorical_accuracy: 0.6154 - val_loss: 6.3794 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 9/30
45/45 [==============================] - ETA: 0s - loss: 0.8369 - categorical_accuracy: 0.6591
Epoch 9: saving model to model_init_2024-03-0411_42_54.002711/model-00009-0.83694-0.65913-6.36008-0.31000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.8369 - categorical_accuracy: 0.6591 - val_loss: 6.3601 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 10/30
45/45 [==============================] - ETA: 0s - loss: 0.7112 - categorical_accuracy: 0.7119
Epoch 10: saving model to model_init_2024-03-0411_42_54.002711/model-00010-0.71125-0.71192-4.15338-0.35000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.7112 - categorical_accuracy: 0.7119 - val_loss: 4.1534 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 11/30
45/45 [==============================] - ETA: 0s - loss: 0.5966 - categorical_accuracy: 0.7707
Epoch 11: saving model to model_init_2024-03-0411_42_54.002711/model-00011-0.59660-0.77074-2.90437-0.31000.h5
45/45 [==============================] - 74s 2s/step - loss: 0.5966 - categorical_accuracy: 0.7707 - val_loss: 2.9044 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 12/30
45/45 [==============================] - ETA: 0s - loss: 0.5585 - categorical_accuracy: 0.7903
Epoch 12: saving model to model_init_2024-03-0411_42_54.002711/model-00012-0.55846-0.79035-1.91452-0.41000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.5585 - categorical_accuracy: 0.7903 - val_loss: 1.9145 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 13/30
45/45 [==============================] - ETA: 0s - loss: 0.4934 - categorical_accuracy: 0.7979
Epoch 13: saving model to model_init_2024-03-0411_42_54.002711/model-00013-0.49344-0.79789-2.29538-0.36000.h5
45/45 [==============================] - 74s 2s/step - loss: 0.4934 - categorical_accuracy: 0.7979 - val_loss: 2.2954 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 14/30
45/45 [==============================] - ETA: 0s - loss: 0.4188 - categorical_accuracy: 0.8311
Epoch 14: saving model to model_init_2024-03-0411_42_54.002711/model-00014-0.41882-0.83107-0.71817-0.75000.h5
45/45 [==============================] - 75s 2s/step - loss: 0.4188 - categorical_accuracy: 0.8311 - val_loss: 0.7182 - val_categorical_accuracy: 0.7500 - lr: 0.0010
Epoch 15/30
45/45 [==============================] - ETA: 0s - loss: 0.3732 - categorical_accuracy: 0.8643
Epoch 15: saving model to model_init_2024-03-0411_42_54.002711/model-00015-0.37322-0.86425-0.71354-0.69000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.3732 - categorical_accuracy: 0.8643 - val_loss: 0.7135 - val_categorical_accuracy: 0.6900 - lr: 0.0010
Epoch 16/30
45/45 [==============================] - ETA: 0s - loss: 0.3602 - categorical_accuracy: 0.8627
Epoch 16: saving model to model_init_2024-03-0411_42_54.002711/model-00016-0.36020-0.86275-1.79631-0.47000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.3602 - categorical_accuracy: 0.8627 - val_loss: 1.7963 - val_categorical_accuracy: 0.4700 - lr: 0.0010
Epoch 17/30
45/45 [==============================] - ETA: 0s - loss: 0.4061 - categorical_accuracy: 0.8492
Epoch 17: saving model to model_init_2024-03-0411_42_54.002711/model-00017-0.40614-0.84917-0.69133-0.78000.h5
45/45 [==============================] - 80s 2s/step - loss: 0.4061 - categorical_accuracy: 0.8492 - val_loss: 0.6913 - val_categorical_accuracy: 0.7800 - lr: 0.0010
Epoch 18/30
45/45 [==============================] - ETA: 0s - loss: 0.4099 - categorical_accuracy: 0.8462
Epoch 18: saving model to model_init_2024-03-0411_42_54.002711/model-00018-0.40991-0.84615-1.56948-0.52000.h5
45/45 [==============================] - 72s 2s/step - loss: 0.4099 - categorical_accuracy: 0.8462 - val_loss: 1.5695 - val_categorical_accuracy: 0.5200 - lr: 0.0010
Epoch 19/30
45/45 [==============================] - ETA: 0s - loss: 0.2472 - categorical_accuracy: 0.8974
Epoch 19: saving model to model_init_2024-03-0411_42_54.002711/model-00019-0.24718-0.89744-0.64426-0.83000.h5
45/45 [==============================] - 78s 2s/step - loss: 0.2472 - categorical_accuracy: 0.8974 - val_loss: 0.6443 - val_categorical_accuracy: 0.8300 - lr: 0.0010
Epoch 20/30
45/45 [==============================] - ETA: 0s - loss: 0.3399 - categorical_accuracy: 0.8824
Epoch 20: saving model to model_init_2024-03-0411_42_54.002711/model-00020-0.33989-0.88235-0.80026-0.76000.h5
45/45 [==============================] - 77s 2s/step - loss: 0.3399 - categorical_accuracy: 0.8824 - val_loss: 0.8003 - val_categorical_accuracy: 0.7600 - lr: 0.0010
Epoch 21/30
45/45 [==============================] - ETA: 0s - loss: 0.2576 - categorical_accuracy: 0.9095
Epoch 21: saving model to model_init_2024-03-0411_42_54.002711/model-00021-0.25764-0.90950-1.65360-0.67000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.2576 - categorical_accuracy: 0.9095 - val_loss: 1.6536 - val_categorical_accuracy: 0.6700 - lr: 0.0010
Epoch 22/30
45/45 [==============================] - ETA: 0s - loss: 0.2929 - categorical_accuracy: 0.8733
Epoch 22: saving model to model_init_2024-03-0411_42_54.002711/model-00022-0.29288-0.87330-1.22758-0.65000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.2929 - categorical_accuracy: 0.8733 - val_loss: 1.2276 - val_categorical_accuracy: 0.6500 - lr: 0.0010
Epoch 23/30
45/45 [==============================] - ETA: 0s - loss: 0.2759 - categorical_accuracy: 0.8959
Epoch 23: saving model to model_init_2024-03-0411_42_54.002711/model-00023-0.27594-0.89593-0.83632-0.67000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.2759 - categorical_accuracy: 0.8959 - val_loss: 0.8363 - val_categorical_accuracy: 0.6700 - lr: 0.0010
Epoch 24/30
45/45 [==============================] - ETA: 0s - loss: 0.3141 - categorical_accuracy: 0.9035
Epoch 24: saving model to model_init_2024-03-0411_42_54.002711/model-00024-0.31413-0.90347-1.50733-0.62000.h5
45/45 [==============================] - 75s 2s/step - loss: 0.3141 - categorical_accuracy: 0.9035 - val_loss: 1.5073 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 25/30
45/45 [==============================] - ETA: 0s - loss: 0.2627 - categorical_accuracy: 0.9065
Epoch 25: saving model to model_init_2024-03-0411_42_54.002711/model-00025-0.26270-0.90649-0.70348-0.76000.h5
45/45 [==============================] - 76s 2s/step - loss: 0.2627 - categorical_accuracy: 0.9065 - val_loss: 0.7035 - val_categorical_accuracy: 0.7600 - lr: 0.0010
Epoch 26/30
45/45 [==============================] - ETA: 0s - loss: 0.2824 - categorical_accuracy: 0.9050
Epoch 26: saving model to model_init_2024-03-0411_42_54.002711/model-00026-0.28244-0.90498-0.94104-0.69000.h5
45/45 [==============================] - 70s 2s/step - loss: 0.2824 - categorical_accuracy: 0.9050 - val_loss: 0.9410 - val_categorical_accuracy: 0.6900 - lr: 0.0010
Epoch 27/30
45/45 [==============================] - ETA: 0s - loss: 0.2600 - categorical_accuracy: 0.9050
Epoch 27: saving model to model_init_2024-03-0411_42_54.002711/model-00027-0.25995-0.90498-0.73749-0.74000.h5
45/45 [==============================] - 62s 1s/step - loss: 0.2600 - categorical_accuracy: 0.9050 - val_loss: 0.7375 - val_categorical_accuracy: 0.7400 - lr: 0.0010
Epoch 28/30
45/45 [==============================] - ETA: 0s - loss: 0.3087 - categorical_accuracy: 0.8673
Epoch 28: saving model to model_init_2024-03-0411_42_54.002711/model-00028-0.30872-0.86727-1.63807-0.61000.h5
45/45 [==============================] - 69s 2s/step - loss: 0.3087 - categorical_accuracy: 0.8673 - val_loss: 1.6381 - val_categorical_accuracy: 0.6100 - lr: 0.0010
Epoch 29/30
45/45 [==============================] - ETA: 0s - loss: 0.2492 - categorical_accuracy: 0.9201
Epoch 29: saving model to model_init_2024-03-0411_42_54.002711/model-00029-0.24916-0.92006-1.87546-0.57000.h5
45/45 [==============================] - 71s 2s/step - loss: 0.2492 - categorical_accuracy: 0.9201 - val_loss: 1.8755 - val_categorical_accuracy: 0.5700 - lr: 0.0010
Epoch 30/30
45/45 [==============================] - ETA: 0s - loss: 0.2306 - categorical_accuracy: 0.9155
Epoch 30: saving model to model_init_2024-03-0411_42_54.002711/model-00030-0.23061-0.91554-1.90569-0.66000.h5
45/45 [==============================] - 72s 2s/step - loss: 0.2306 - categorical_accuracy: 0.9155 - val_loss: 1.9057 - val_categorical_accuracy: 0.6600 - lr: 0.0010

# Model 5 - Batch size 20, epochs 30, drop out 0.4 and dense 64

In [31]:
batch_size = 20
num_epochs = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [33]:
model_5 = generate_model(batch_size, img_height, img_width, 0.4, 64)

In [34]:
optimiser = 'adam' #write your optimizer
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_4.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 15, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_12 (Activation)  (None, 15, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_18 (Ba  (None, 15, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 max_pooling3d_12 (MaxPooli  (None, 7, 60, 60, 16)     0         
 ng3D)                                                           
                                                                 
 conv3d_13 (Conv3D)          (None, 7, 60, 60, 32)    

In [35]:
#model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
33/34 [============================>.] - ETA: 2s - loss: 1.8639 - categorical_accuracy: 0.3530
/tmp/ipykernel_34/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
34/34 [==============================] - ETA: 0s - loss: 1.8634 - categorical_accuracy: 0.3544Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 20

Epoch 1: saving model to model_init_2024-03-0411_42_54.002711/model-00001-1.86336-0.35445-2.24560-0.16000.h5
34/34 [==============================] - 86s 2s/step - loss: 1.8634 - categorical_accuracy: 0.3544 - val_loss: 2.2456 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 2/30
34/34 [==============================] - ETA: 0s - loss: 1.3272 - categorical_accuracy: 0.5053
Epoch 2: saving model to model_init_2024-03-0411_42_54.002711/model-00002-1.32720-0.50528-3.23536-0.13000.h5
34/34 [==============================] - 78s 2s/step - loss: 1.3272 - categorical_accuracy: 0.5053 - val_loss: 3.2354 - val_categorical_accuracy: 0.1300 - lr: 0.0010
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 1.2342 - categorical_accuracy: 0.5204
Epoch 3: saving model to model_init_2024-03-0411_42_54.002711/model-00003-1.23419-0.52036-3.57203-0.16000.h5
34/34 [==============================] - 74s 2s/step - loss: 1.2342 - categorical_accuracy: 0.5204 - val_loss: 3.5720 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 4/30
34/34 [==============================] - ETA: 0s - loss: 1.0491 - categorical_accuracy: 0.5822
Epoch 4: saving model to model_init_2024-03-0411_42_54.002711/model-00004-1.04908-0.58220-4.62160-0.20000.h5
34/34 [==============================] - 69s 2s/step - loss: 1.0491 - categorical_accuracy: 0.5822 - val_loss: 4.6216 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 5/30
34/34 [==============================] - ETA: 0s - loss: 1.0077 - categorical_accuracy: 0.6365
Epoch 5: saving model to model_init_2024-03-0411_42_54.002711/model-00005-1.00768-0.63650-6.09362-0.22000.h5
34/34 [==============================] - 71s 2s/step - loss: 1.0077 - categorical_accuracy: 0.6365 - val_loss: 6.0936 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 6/30
34/34 [==============================] - ETA: 0s - loss: 0.9764 - categorical_accuracy: 0.6259
Epoch 6: saving model to model_init_2024-03-0411_42_54.002711/model-00006-0.97641-0.62594-8.90670-0.21000.h5
34/34 [==============================] - 72s 2s/step - loss: 0.9764 - categorical_accuracy: 0.6259 - val_loss: 8.9067 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 7/30
34/34 [==============================] - ETA: 0s - loss: 0.8302 - categorical_accuracy: 0.6938
Epoch 7: saving model to model_init_2024-03-0411_42_54.002711/model-00007-0.83020-0.69382-7.77892-0.19000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.8302 - categorical_accuracy: 0.6938 - val_loss: 7.7789 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 8/30
34/34 [==============================] - ETA: 0s - loss: 0.7492 - categorical_accuracy: 0.7059
Epoch 8: saving model to model_init_2024-03-0411_42_54.002711/model-00008-0.74916-0.70588-5.89156-0.16000.h5
34/34 [==============================] - 74s 2s/step - loss: 0.7492 - categorical_accuracy: 0.7059 - val_loss: 5.8916 - val_categorical_accuracy: 0.1600 - lr: 0.0010
Epoch 9/30
34/34 [==============================] - ETA: 0s - loss: 0.6672 - categorical_accuracy: 0.7526
Epoch 9: saving model to model_init_2024-03-0411_42_54.002711/model-00009-0.66724-0.75264-5.42276-0.20000.h5
34/34 [==============================] - 66s 2s/step - loss: 0.6672 - categorical_accuracy: 0.7526 - val_loss: 5.4228 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 10/30
34/34 [==============================] - ETA: 0s - loss: 0.6035 - categorical_accuracy: 0.7692
Epoch 10: saving model to model_init_2024-03-0411_42_54.002711/model-00010-0.60347-0.76923-5.15601-0.22000.h5
34/34 [==============================] - 71s 2s/step - loss: 0.6035 - categorical_accuracy: 0.7692 - val_loss: 5.1560 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 11/30
34/34 [==============================] - ETA: 0s - loss: 0.4734 - categorical_accuracy: 0.8220
Epoch 11: saving model to model_init_2024-03-0411_42_54.002711/model-00011-0.47338-0.82202-4.79484-0.34000.h5
34/34 [==============================] - 75s 2s/step - loss: 0.4734 - categorical_accuracy: 0.8220 - val_loss: 4.7948 - val_categorical_accuracy: 0.3400 - lr: 0.0010
Epoch 12/30
34/34 [==============================] - ETA: 0s - loss: 0.4311 - categorical_accuracy: 0.8431
Epoch 12: saving model to model_init_2024-03-0411_42_54.002711/model-00012-0.43114-0.84314-4.13371-0.31000.h5
34/34 [==============================] - 64s 2s/step - loss: 0.4311 - categorical_accuracy: 0.8431 - val_loss: 4.1337 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 13/30
34/34 [==============================] - ETA: 0s - loss: 0.4908 - categorical_accuracy: 0.8115
Epoch 13: saving model to model_init_2024-03-0411_42_54.002711/model-00013-0.49082-0.81146-4.45952-0.24000.h5
34/34 [==============================] - 71s 2s/step - loss: 0.4908 - categorical_accuracy: 0.8115 - val_loss: 4.4595 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 14/30
34/34 [==============================] - ETA: 0s - loss: 0.6516 - categorical_accuracy: 0.7541
Epoch 14: saving model to model_init_2024-03-0411_42_54.002711/model-00014-0.65159-0.75415-4.14772-0.22000.h5
34/34 [==============================] - 71s 2s/step - loss: 0.6516 - categorical_accuracy: 0.7541 - val_loss: 4.1477 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 15/30
34/34 [==============================] - ETA: 0s - loss: 0.7039 - categorical_accuracy: 0.7360
Epoch 15: saving model to model_init_2024-03-0411_42_54.002711/model-00015-0.70392-0.73605-2.20259-0.41000.h5
34/34 [==============================] - 71s 2s/step - loss: 0.7039 - categorical_accuracy: 0.7360 - val_loss: 2.2026 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 16/30
34/34 [==============================] - ETA: 0s - loss: 0.5129 - categorical_accuracy: 0.8190
Epoch 16: saving model to model_init_2024-03-0411_42_54.002711/model-00016-0.51294-0.81900-2.57291-0.33000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.5129 - categorical_accuracy: 0.8190 - val_loss: 2.5729 - val_categorical_accuracy: 0.3300 - lr: 0.0010
Epoch 17/30
34/34 [==============================] - ETA: 0s - loss: 0.4920 - categorical_accuracy: 0.8130
Epoch 17: saving model to model_init_2024-03-0411_42_54.002711/model-00017-0.49201-0.81297-1.29582-0.55000.h5
34/34 [==============================] - 70s 2s/step - loss: 0.4920 - categorical_accuracy: 0.8130 - val_loss: 1.2958 - val_categorical_accuracy: 0.5500 - lr: 0.0010
Epoch 18/30
34/34 [==============================] - ETA: 0s - loss: 0.4520 - categorical_accuracy: 0.8446
Epoch 18: saving model to model_init_2024-03-0411_42_54.002711/model-00018-0.45202-0.84465-1.68900-0.45000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.4520 - categorical_accuracy: 0.8446 - val_loss: 1.6890 - val_categorical_accuracy: 0.4500 - lr: 0.0010
Epoch 19/30
34/34 [==============================] - ETA: 0s - loss: 0.3806 - categorical_accuracy: 0.8597
Epoch 19: saving model to model_init_2024-03-0411_42_54.002711/model-00019-0.38064-0.85973-1.11402-0.64000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.3806 - categorical_accuracy: 0.8597 - val_loss: 1.1140 - val_categorical_accuracy: 0.6400 - lr: 0.0010
Epoch 20/30
34/34 [==============================] - ETA: 0s - loss: 0.3680 - categorical_accuracy: 0.8718
Epoch 20: saving model to model_init_2024-03-0411_42_54.002711/model-00020-0.36801-0.87179-1.99041-0.48000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.3680 - categorical_accuracy: 0.8718 - val_loss: 1.9904 - val_categorical_accuracy: 0.4800 - lr: 0.0010
Epoch 21/30
34/34 [==============================] - ETA: 0s - loss: 0.5364 - categorical_accuracy: 0.8024
Epoch 21: saving model to model_init_2024-03-0411_42_54.002711/model-00021-0.53638-0.80241-2.12437-0.50000.h5
34/34 [==============================] - 68s 2s/step - loss: 0.5364 - categorical_accuracy: 0.8024 - val_loss: 2.1244 - val_categorical_accuracy: 0.5000 - lr: 0.0010
Epoch 22/30
34/34 [==============================] - ETA: 0s - loss: 0.4295 - categorical_accuracy: 0.8326
Epoch 22: saving model to model_init_2024-03-0411_42_54.002711/model-00022-0.42949-0.83258-3.24038-0.36000.h5
34/34 [==============================] - 68s 2s/step - loss: 0.4295 - categorical_accuracy: 0.8326 - val_loss: 3.2404 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 23/30
34/34 [==============================] - ETA: 0s - loss: 0.3869 - categorical_accuracy: 0.8597
Epoch 23: saving model to model_init_2024-03-0411_42_54.002711/model-00023-0.38687-0.85973-1.82410-0.47000.h5
34/34 [==============================] - 70s 2s/step - loss: 0.3869 - categorical_accuracy: 0.8597 - val_loss: 1.8241 - val_categorical_accuracy: 0.4700 - lr: 0.0010
Epoch 24/30
34/34 [==============================] - ETA: 0s - loss: 0.4450 - categorical_accuracy: 0.8326
Epoch 24: saving model to model_init_2024-03-0411_42_54.002711/model-00024-0.44500-0.83258-1.26690-0.56000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.4450 - categorical_accuracy: 0.8326 - val_loss: 1.2669 - val_categorical_accuracy: 0.5600 - lr: 0.0010
Epoch 25/30
34/34 [==============================] - ETA: 0s - loss: 0.3500 - categorical_accuracy: 0.8763
Epoch 25: saving model to model_init_2024-03-0411_42_54.002711/model-00025-0.35001-0.87632-1.01419-0.62000.h5
34/34 [==============================] - 68s 2s/step - loss: 0.3500 - categorical_accuracy: 0.8763 - val_loss: 1.0142 - val_categorical_accuracy: 0.6200 - lr: 0.0010
Epoch 26/30
34/34 [==============================] - ETA: 0s - loss: 0.2726 - categorical_accuracy: 0.9020
Epoch 26: saving model to model_init_2024-03-0411_42_54.002711/model-00026-0.27263-0.90196-0.69602-0.77000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.2726 - categorical_accuracy: 0.9020 - val_loss: 0.6960 - val_categorical_accuracy: 0.7700 - lr: 0.0010
Epoch 27/30
34/34 [==============================] - ETA: 0s - loss: 0.2320 - categorical_accuracy: 0.9246
Epoch 27: saving model to model_init_2024-03-0411_42_54.002711/model-00027-0.23203-0.92459-0.63130-0.73000.h5
34/34 [==============================] - 70s 2s/step - loss: 0.2320 - categorical_accuracy: 0.9246 - val_loss: 0.6313 - val_categorical_accuracy: 0.7300 - lr: 0.0010
Epoch 28/30
34/34 [==============================] - ETA: 0s - loss: 0.2110 - categorical_accuracy: 0.9321
Epoch 28: saving model to model_init_2024-03-0411_42_54.002711/model-00028-0.21095-0.93213-0.69152-0.74000.h5
34/34 [==============================] - 70s 2s/step - loss: 0.2110 - categorical_accuracy: 0.9321 - val_loss: 0.6915 - val_categorical_accuracy: 0.7400 - lr: 0.0010
Epoch 29/30
34/34 [==============================] - ETA: 0s - loss: 0.2044 - categorical_accuracy: 0.9276
Epoch 29: saving model to model_init_2024-03-0411_42_54.002711/model-00029-0.20443-0.92760-0.64262-0.78000.h5
34/34 [==============================] - 71s 2s/step - loss: 0.2044 - categorical_accuracy: 0.9276 - val_loss: 0.6426 - val_categorical_accuracy: 0.7800 - lr: 0.0010
Epoch 30/30
34/34 [==============================] - ETA: 0s - loss: 0.1663 - categorical_accuracy: 0.9487
Epoch 30: saving model to model_init_2024-03-0411_42_54.002711/model-00030-0.16627-0.94872-0.88465-0.61000.h5
34/34 [==============================] - 69s 2s/step - loss: 0.1663 - categorical_accuracy: 0.9487 - val_loss: 0.8847 - val_categorical_accuracy: 0.6100 - lr: 0.0010
<keras.src.callbacks.History at 0x7c84b07b76d0>

# Model 6 - Batch size 20, epochs 30, drop out 0.4 and dense 64

In [36]:
batch_size = 20
num_epochs = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [37]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [38]:
model_6 = generate_model(batch_size, img_height, img_width, 0.4, 128)

In [39]:
optimiser = 'adam' #write your optimizer
model_6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_6.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_20 (Conv3D)          (None, 20, 120, 120, 16   1312      
                             )                                   
                                                                 
 activation_20 (Activation)  (None, 20, 120, 120, 16   0         
                             )                                   
                                                                 
 batch_normalization_30 (Ba  (None, 20, 120, 120, 16   64        
 tchNormalization)           )                                   
                                                                 
 max_pooling3d_20 (MaxPooli  (None, 10, 60, 60, 16)    0         
 ng3D)                                                           
                                                                 
 conv3d_21 (Conv3D)          (None, 10, 60, 60, 32)   

In [ ]:
#model_6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

# Conv 2D + LSTM

In [50]:
from tensorflow.keras.applications import mobilenet

In [46]:

num_epochs = 30
batch_size = 20

In [51]:
mobilenet = mobilenet.MobileNet(weights='imagenet', include_top=False)


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model7 = Sequential([
    TimeDistributed(mobilenet, input_shape=(batch_size ,img_height, img_width,3))
], name="mobilenet_lstm")

for layer in model7.layers:
    layer.trainable = False

model7.add(TimeDistributed(BatchNormalization()))
model7.add(TimeDistributed(MaxPooling2D(tuple([2]*2))))
model7.add(TimeDistributed(Flatten()))

model7.add(LSTM(256))
model7.add(Dropout(0.2))

model7.add(Dense(256,activation='relu'))
model7.add(Dropout(0.2))

model7.add(Dense(5, activation='softmax'))

In [52]:
model7.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model7.summary())

Model: "mobilenet_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDi  (None, 20, 3, 3, 1024)    3228864   
 stributed)                                                      
                                                                 
 time_distributed_7 (TimeDi  (None, 20, 3, 3, 1024)    4096      
 stributed)                                                      
                                                                 
 time_distributed_8 (TimeDi  (None, 20, 1, 1, 1024)    0         
 stributed)                                                      
                                                                 
 time_distributed_9 (TimeDi  (None, 20, 1024)          0         
 stributed)                                                      
                                                                 
 lstm (LSTM)                 (None, 256)            

In [53]:
#model7_history = model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
            callbacks=callbacks_list, validation_data=val_generator, 
            validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_33/55885017.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model7_history = model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
/tmp/ipykernel_33/515647629.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  /kaggle/input/gesture-reconition/Project_data/train ; batch size = 20
Epoch 1/30
32/34 [===========================>..] - ETA: 6s - loss: 1.6575 - categorical_accuracy: 0.2078

/tmp/ipykernel_33/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


34/34 [==============================] - ETA: 0s - loss: 1.6536 - categorical_accuracy: 0.2097Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 20

Epoch 1: saving model to model_init_2024-03-0511_47_48.329730/model-00001-1.65357-0.20965-1.42314-0.56000.h5
34/34 [==============================] - 124s 4s/step - loss: 1.6536 - categorical_accuracy: 0.2097 - val_loss: 1.4231 - val_categorical_accuracy: 0.5600 - lr: 0.0010
Epoch 2/30


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - ETA: 0s - loss: 1.2528 - categorical_accuracy: 0.4932
Epoch 2: saving model to model_init_2024-03-0511_47_48.329730/model-00002-1.25284-0.49321-1.12043-0.52000.h5
34/34 [==============================] - 101s 3s/step - loss: 1.2528 - categorical_accuracy: 0.4932 - val_loss: 1.1204 - val_categorical_accuracy: 0.5200 - lr: 0.0010
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 0.8916 - categorical_accuracy: 0.6576
Epoch 3: saving model to model_init_2024-03-0511_47_48.329730/model-00003-0.89160-0.65762-0.93802-0.60000.h5
34/34 [==============================] - 95s 3s/step - loss: 0.8916 - categorical_accuracy: 0.6576 - val_loss: 0.9380 - val_categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 4/30
34/34 [==============================] - ETA: 0s - loss: 0.6509 - categorical_accuracy: 0.7692
Epoch 4: saving model to model_init_2024-03-0511_47_48.329730/model-00004-0.65092-0.76923-0.89552-0.64000.h5
34/34 [==========================

Source path =  /kaggle/input/gesture-reconition/Project_data/train ; batch size = 20
Epoch 1/30
32/34 [===========================>..] - ETA: 6s - loss: 1.6575 - categorical_accuracy: 0.2078
/tmp/ipykernel_33/515647629.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
34/34 [==============================] - ETA: 0s - loss: 1.6536 - categorical_accuracy: 0.2097Source path =  /kaggle/input/gesture-reconition/Project_data/val ; batch size = 20

Epoch 1: saving model to model_init_2024-03-0511_47_48.329730/model-00001-1.65357-0.20965-1.42314-0.56000.h5
34/34 [==============================] - 124s 4s/step - loss: 1.6536 - categorical_accuracy: 0.2097 - val_loss: 1.4231 - val_categorical_accuracy: 0.5600 - lr: 0.0010
Epoch 2/30
/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
34/34 [==============================] - ETA: 0s - loss: 1.2528 - categorical_accuracy: 0.4932
Epoch 2: saving model to model_init_2024-03-0511_47_48.329730/model-00002-1.25284-0.49321-1.12043-0.52000.h5
34/34 [==============================] - 101s 3s/step - loss: 1.2528 - categorical_accuracy: 0.4932 - val_loss: 1.1204 - val_categorical_accuracy: 0.5200 - lr: 0.0010
Epoch 3/30
34/34 [==============================] - ETA: 0s - loss: 0.8916 - categorical_accuracy: 0.6576
Epoch 3: saving model to model_init_2024-03-0511_47_48.329730/model-00003-0.89160-0.65762-0.93802-0.60000.h5
34/34 [==============================] - 95s 3s/step - loss: 0.8916 - categorical_accuracy: 0.6576 - val_loss: 0.9380 - val_categorical_accuracy: 0.6000 - lr: 0.0010
Epoch 4/30
34/34 [==============================] - ETA: 0s - loss: 0.6509 - categorical_accuracy: 0.7692
Epoch 4: saving model to model_init_2024-03-0511_47_48.329730/model-00004-0.65092-0.76923-0.89552-0.64000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.6509 - categorical_accuracy: 0.7692 - val_loss: 0.8955 - val_categorical_accuracy: 0.6400 - lr: 0.0010
Epoch 5/30
34/34 [==============================] - ETA: 0s - loss: 0.5587 - categorical_accuracy: 0.7934
Epoch 5: saving model to model_init_2024-03-0511_47_48.329730/model-00005-0.55874-0.79336-0.84706-0.63000.h5
34/34 [==============================] - 91s 3s/step - loss: 0.5587 - categorical_accuracy: 0.7934 - val_loss: 0.8471 - val_categorical_accuracy: 0.6300 - lr: 0.0010
Epoch 6/30
34/34 [==============================] - ETA: 0s - loss: 0.4528 - categorical_accuracy: 0.8326
Epoch 6: saving model to model_init_2024-03-0511_47_48.329730/model-00006-0.45282-0.83258-0.83271-0.75000.h5
34/34 [==============================] - 95s 3s/step - loss: 0.4528 - categorical_accuracy: 0.8326 - val_loss: 0.8327 - val_categorical_accuracy: 0.7500 - lr: 0.0010
Epoch 7/30
34/34 [==============================] - ETA: 0s - loss: 0.3826 - categorical_accuracy: 0.8507
Epoch 7: saving model to model_init_2024-03-0511_47_48.329730/model-00007-0.38256-0.85068-0.98025-0.75000.h5
34/34 [==============================] - 89s 3s/step - loss: 0.3826 - categorical_accuracy: 0.8507 - val_loss: 0.9802 - val_categorical_accuracy: 0.7500 - lr: 0.0010
Epoch 8/30
34/34 [==============================] - ETA: 0s - loss: 0.3788 - categorical_accuracy: 0.8658
Epoch 8: saving model to model_init_2024-03-0511_47_48.329730/model-00008-0.37882-0.86576-0.89857-0.74000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.3788 - categorical_accuracy: 0.8658 - val_loss: 0.8986 - val_categorical_accuracy: 0.7400 - lr: 0.0010
Epoch 9/30
34/34 [==============================] - ETA: 0s - loss: 0.2979 - categorical_accuracy: 0.8929
Epoch 9: saving model to model_init_2024-03-0511_47_48.329730/model-00009-0.29795-0.89291-0.86895-0.70000.h5
34/34 [==============================] - 91s 3s/step - loss: 0.2979 - categorical_accuracy: 0.8929 - val_loss: 0.8690 - val_categorical_accuracy: 0.7000 - lr: 0.0010
Epoch 10/30
34/34 [==============================] - ETA: 0s - loss: 0.2245 - categorical_accuracy: 0.9216
Epoch 10: saving model to model_init_2024-03-0511_47_48.329730/model-00010-0.22453-0.92157-1.03054-0.73000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.2245 - categorical_accuracy: 0.9216 - val_loss: 1.0305 - val_categorical_accuracy: 0.7300 - lr: 0.0010
Epoch 11/30
34/34 [==============================] - ETA: 0s - loss: 0.1593 - categorical_accuracy: 0.9563
Epoch 11: saving model to model_init_2024-03-0511_47_48.329730/model-00011-0.15928-0.95626-0.81671-0.72000.h5
34/34 [==============================] - 93s 3s/step - loss: 0.1593 - categorical_accuracy: 0.9563 - val_loss: 0.8167 - val_categorical_accuracy: 0.7200 - lr: 0.0010
Epoch 12/30
34/34 [==============================] - ETA: 0s - loss: 0.2377 - categorical_accuracy: 0.9080
Epoch 12: saving model to model_init_2024-03-0511_47_48.329730/model-00012-0.23773-0.90799-0.99645-0.71000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.2377 - categorical_accuracy: 0.9080 - val_loss: 0.9965 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 13/30
34/34 [==============================] - ETA: 0s - loss: 0.1370 - categorical_accuracy: 0.9653
Epoch 13: saving model to model_init_2024-03-0511_47_48.329730/model-00013-0.13696-0.96531-1.08947-0.68000.h5
34/34 [==============================] - 93s 3s/step - loss: 0.1370 - categorical_accuracy: 0.9653 - val_loss: 1.0895 - val_categorical_accuracy: 0.6800 - lr: 0.0010
Epoch 14/30
34/34 [==============================] - ETA: 0s - loss: 0.1445 - categorical_accuracy: 0.9593
Epoch 14: saving model to model_init_2024-03-0511_47_48.329730/model-00014-0.14445-0.95928-1.01710-0.72000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.1445 - categorical_accuracy: 0.9593 - val_loss: 1.0171 - val_categorical_accuracy: 0.7200 - lr: 0.0010
Epoch 15/30
34/34 [==============================] - ETA: 0s - loss: 0.1185 - categorical_accuracy: 0.9578
Epoch 15: saving model to model_init_2024-03-0511_47_48.329730/model-00015-0.11848-0.95777-1.12338-0.70000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.1185 - categorical_accuracy: 0.9578 - val_loss: 1.1234 - val_categorical_accuracy: 0.7000 - lr: 0.0010
Epoch 16/30
34/34 [==============================] - ETA: 0s - loss: 0.0814 - categorical_accuracy: 0.9713
Epoch 16: saving model to model_init_2024-03-0511_47_48.329730/model-00016-0.08139-0.97134-1.28595-0.70000.h5
34/34 [==============================] - 93s 3s/step - loss: 0.0814 - categorical_accuracy: 0.9713 - val_loss: 1.2859 - val_categorical_accuracy: 0.7000 - lr: 0.0010
Epoch 17/30
34/34 [==============================] - ETA: 0s - loss: 0.0559 - categorical_accuracy: 0.9789
Epoch 17: saving model to model_init_2024-03-0511_47_48.329730/model-00017-0.05590-0.97888-1.21287-0.74000.h5
34/34 [==============================] - 92s 3s/step - loss: 0.0559 - categorical_accuracy: 0.9789 - val_loss: 1.2129 - val_categorical_accuracy: 0.7400 - lr: 0.0010
Epoch 18/30
34/34 [==============================] - ETA: 0s - loss: 0.0677 - categorical_accuracy: 0.9744
Epoch 18: saving model to model_init_2024-03-0511_47_48.329730/model-00018-0.06770-0.97436-0.95593-0.76000.h5
34/34 [==============================] - 93s 3s/step - loss: 0.0677 - categorical_accuracy: 0.9744 - val_loss: 0.9559 - val_categorical_accuracy: 0.7600 - lr: 0.0010
Epoch 19/30
34/34 [==============================] - ETA: 0s - loss: 0.1202 - categorical_accuracy: 0.9623
Epoch 19: saving model to model_init_2024-03-0511_47_48.329730/model-00019-0.12019-0.96229-1.36383-0.70000.h5
34/34 [==============================] - 96s 3s/step - loss: 0.1202 - categorical_accuracy: 0.9623 - val_loss: 1.3638 - val_categorical_accuracy: 0.7000 - lr: 0.0010
Epoch 20/30
34/34 [==============================] - ETA: 0s - loss: 0.0716 - categorical_accuracy: 0.9819
Epoch 20: saving model to model_init_2024-03-0511_47_48.329730/model-00020-0.07159-0.98190-0.94132-0.77000.h5
34/34 [==============================] - 95s 3s/step - loss: 0.0716 - categorical_accuracy: 0.9819 - val_loss: 0.9413 - val_categorical_accuracy: 0.7700 - lr: 0.0010
Epoch 21/30
34/34 [==============================] - ETA: 0s - loss: 0.0767 - categorical_accuracy: 0.9744
Epoch 21: saving model to model_init_2024-03-0511_47_48.329730/model-00021-0.07665-0.97436-1.04564-0.68000.h5
34/34 [==============================] - 96s 3s/step - loss: 0.0767 - categorical_accuracy: 0.9744 - val_loss: 1.0456 - val_categorical_accuracy: 0.6800 - lr: 0.0010
Epoch 22/30
34/34 [==============================] - ETA: 0s - loss: 0.0671 - categorical_accuracy: 0.9759
Epoch 22: saving model to model_init_2024-03-0511_47_48.329730/model-00022-0.06714-0.97587-1.18532-0.68000.h5
34/34 [==============================] - 95s 3s/step - loss: 0.0671 - categorical_accuracy: 0.9759 - val_loss: 1.1853 - val_categorical_accuracy: 0.6800 - lr: 0.0010
Epoch 23/30
34/34 [==============================] - ETA: 0s - loss: 0.0488 - categorical_accuracy: 0.9819
Epoch 23: saving model to model_init_2024-03-0511_47_48.329730/model-00023-0.04879-0.98190-1.46148-0.63000.h5
34/34 [==============================] - 96s 3s/step - loss: 0.0488 - categorical_accuracy: 0.9819 - val_loss: 1.4615 - val_categorical_accuracy: 0.6300 - lr: 0.0010
Epoch 24/30
34/34 [==============================] - ETA: 0s - loss: 0.0834 - categorical_accuracy: 0.9698
Epoch 24: saving model to model_init_2024-03-0511_47_48.329730/model-00024-0.08341-0.96983-1.14717-0.71000.h5
34/34 [==============================] - 93s 3s/step - loss: 0.0834 - categorical_accuracy: 0.9698 - val_loss: 1.1472 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 25/30
34/34 [==============================] - ETA: 0s - loss: 0.0627 - categorical_accuracy: 0.9759
Epoch 25: saving model to model_init_2024-03-0511_47_48.329730/model-00025-0.06267-0.97587-1.12629-0.68000.h5
34/34 [==============================] - 94s 3s/step - loss: 0.0627 - categorical_accuracy: 0.9759 - val_loss: 1.1263 - val_categorical_accuracy: 0.6800 - lr: 0.0010
Epoch 26/30
34/34 [==============================] - ETA: 0s - loss: 0.0813 - categorical_accuracy: 0.9744
Epoch 26: saving model to model_init_2024-03-0511_47_48.329730/model-00026-0.08130-0.97436-1.22396-0.71000.h5
34/34 [==============================] - 139s 4s/step - loss: 0.0813 - categorical_accuracy: 0.9744 - val_loss: 1.2240 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 27/30
34/34 [==============================] - ETA: 0s - loss: 0.1307 - categorical_accuracy: 0.9442
Epoch 27: saving model to model_init_2024-03-0511_47_48.329730/model-00027-0.13068-0.94419-1.35618-0.67000.h5
34/34 [==============================] - 99s 3s/step - loss: 0.1307 - categorical_accuracy: 0.9442 - val_loss: 1.3562 - val_categorical_accuracy: 0.6700 - lr: 0.0010
Epoch 28/30
34/34 [==============================] - ETA: 0s - loss: 0.1425 - categorical_accuracy: 0.9472
Epoch 28: saving model to model_init_2024-03-0511_47_48.329730/model-00028-0.14253-0.94721-1.27657-0.71000.h5
34/34 [==============================] - 93s 3s/step - loss: 0.1425 - categorical_accuracy: 0.9472 - val_loss: 1.2766 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 29/30
34/34 [==============================] - ETA: 0s - loss: 0.0571 - categorical_accuracy: 0.9789
Epoch 29: saving model to model_init_2024-03-0511_47_48.329730/model-00029-0.05713-0.97888-1.18389-0.72000.h5
34/34 [==============================] - 117s 3s/step - loss: 0.0571 - categorical_accuracy: 0.9789 - val_loss: 1.1839 - val_categorical_accuracy: 0.7200 - lr: 0.0010
Epoch 30/30
34/34 [==============================] - ETA: 0s - loss: 0.0648 - categorical_accuracy: 0.9759
Epoch 30: saving model to model_init_2024-03-0511_47_48.329730/model-00030-0.06481-0.97587-1.05940-0.76000.h5
34/34 [==============================] - 91s 3s/step - loss: 0.0648 - categorical_accuracy: 0.9759 - val_loss: 1.0594 - val_categorical_accuracy: 0.7600 - lr: 0.0010

**The best Validation accuracy we see is 77%**